**Niks Pille**

All Tickers

In [9]:
#Getting all tickers in a list
All_Tickers = []
for i in range(4521):
    All_Tickers.append(Only_Common_Stocks['symbol'][i])

Creating 'Stock_With_All_Data'

In [171]:
Stocks_With_All_Data = []
for i in range(4521):
    if len(yf.Ticker(All_Tickers[i]).history(start='2000-01-02', end='2020-01-01')[['Close']]) == 5031:
        Stocks_With_All_Data.append(All_Tickers[i])

- AGM.A: No data found, symbol may be delisted
- ALUS: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- AMHC: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- ARQT: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BATL: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BDTX: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BEAM: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BF.A: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BH.A: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- BRK.A: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BSBK: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BWL.A: No data found, symbol may be delisted
- CHPM: Data doesn't exist for startDate = 946767600, endDate 

In [186]:
with open('Stocks_With_Data.csv','w',newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(Stocks_With_All_Data)

Creating 'Three_Month_Treasury_Bill'

In [38]:
#DTB3 is taken from https://fred.stlouisfed.org/
DTB3 = pd.read_csv('DTB3.csv', index_col = 0)
#Using Apple as a benchmark
aapl = yf.Ticker('aapl').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Getting dates from Apple
Dates = []
for i in range(len(aapl)):
    if str(aapl.index[i])[:10] in DTB3.index:
        Dates.append(str(aapl.index[i])[:10])
#Sorting out missing values
Three_Month_Treasury_Bill = DTB3.loc[Dates][DTB3.loc[Dates].DTB3 != '.']
#Creating a csv-file with the Three Month Treasury Bill
Three_Month_Treasury_Bill.to_csv('Three_Month_Treasury_Bill.csv')

Creating 'All_Stocks'

In [45]:
#Make a table with the new dates
Ny = DTB3.loc[Dates]
#Make a new column with strings
Ny['Datoer'] = Dates
#Set the new column as index to make it a callable string
Nyy = Ny.set_index('Datoer')
#A for loop to get all tables for the chosen stocks into a single list, to be used in concatenating
stock = []
for i in list(dict.fromkeys(Stocks)):
    asset = yf.Ticker(i).history(start='2000-01-02', end='2020-01-01')[['Close']]
    asset.columns = [i]
    stock.append(asset)
#Concat all the tables
Concat_Table = pd.concat([i for i in stock], axis=1, join='inner')
#Concat the stocks with the risk free rate
All_Tables = pd.concat([Nyy, Concat_Table.reindex(Nyy.index)],axis=1)
#Save to a csv file
All_Tables.to_csv('All_Stocks.csv')

Creating 'Table'

In [48]:
Everything = pd.read_csv('All_Stocks.csv', index_col=0)
#Russell3000 is used as the market rate since it covers 98% of the American market
Russell_3000 = yf.Ticker('^RUA').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Sorting out missing values from the risk-free rate
gj = Everything.loc[:][Everything.loc[:].DTB3 != '.']
#Making a new date list with the dates from Russell 3000
Datess = []
for i in range(len(Russell_3000)):
    if str(Russell_3000.index[i])[:10] in gj.index:
        Datess.append(str(Russell_3000.index[i])[:10])
#Making the dates strings so they are callable
gh = []
for i in range(len(Russell_3000)):
    gh.append(str(Russell_3000.index[i])[:10])
#Making a new column with the string dates and set it as index
Russell_3000['Datoer'] = gh
gk = Russell_3000.set_index('Datoer')
#Set the name of the column
gk.columns = ['Russell3000']
#Concat Russell3000 to the rest
Table = pd.concat([gj.loc[Datess],gk.loc[Datess]],axis=1)
Table.to_csv('Table.csv')

# Bachelorproject

Documentation:  
https://addisonlynch.github.io/iexfinance/stable/stocks.html  
https://github.com/ranaroussi/yfinance

## Getting Data

In [1]:
#All packages used
from iexfinance.stocks import Stock
from iexfinance.refdata import get_symbols
import yfinance as yf
import numpy as np
import pandas as pd
import csv
token = 'sk_32c5d1b5f6b044f294391ae88eac3e47'

In [2]:
#Get a table with everything on the financial markets in USA
Alle_symboler = get_symbols(output_format='pandas', token=token)
#Sorting out all the non common stocks
Only_Common_Stocks = get_symbols()[get_symbols().type == 'cs'].reset_index(drop=True)

In [53]:
with open('Stocks_With_Data.csv',newline='') as file:
    reader = csv.reader(file)
    Stocks_With_All_Data = list(reader)
#Drawing 300 random stocks for which to be analyzed
Stocks = []
np.random.seed(2020)
for i in range(300):
    Stocks.append(Stocks_With_All_Data[0][np.random.randint(0,len(Stocks_With_All_Data[0]))])

Stocks.sort()
Every_Stock = list(dict.fromkeys(Stocks))

### Adjusting so all data match with no missing values

In [49]:
All = pd.read_csv('Table.csv', index_col=0)

In [55]:
columns = []
for i in range(283):
    columns.append(All.columns[i])
#Creating a table with the log values
Logtable = pd.DataFrame()
for i in range(283):
    Logtable[columns[i]] = All[columns[i]].apply(np.log)

In [84]:
Daily = pd.DataFrame()
for i in range(len(columns)):
    Daily[columns[i]] = np.diff(Logtable[columns[i]],axis=0)

In [85]:
Daily_returns = Daily.set_index(Logtable.index.values[0:-1])
Daily_returns.to_csv('Daily_returns.csv')

In [86]:
Daily_returns

,DTB3,A,AB,ABC,ADM,AEE,AEM,AES,AGM,AIT,...,WTM,WVVI,WWD,WWE,WYY,XLNX,XRAY,XRX,Y,Russell3000
2000-01-03,0.000000,-0.079649,-0.004035,-0.072571,-0.011364,0.000000,-0.026404,-0.040341,-0.026170,-0.015306,...,-0.008499,0.030772,0.002886,-0.043394,0.000000,-0.022990,0.000000,-0.047507,-0.005382,-0.039434
2000-01-04,0.001896,-0.063937,0.004035,0.075898,-0.015839,0.037802,-0.008396,0.010509,-0.054488,-0.023408,...,0.028516,0.029853,-0.026279,0.036521,-0.013628,-0.022278,0.018480,0.050166,-0.003699,0.001705
2000-01-05,-0.005698,-0.038691,0.015979,0.073633,0.005789,-0.003231,0.016723,0.008257,0.037681,-0.018593,...,-0.007308,-0.029853,-0.011905,-0.065551,0.186899,-0.090659,0.000000,-0.020888,0.005402,-0.000476
2000-01-06,-0.005731,0.079939,0.032491,0.121890,0.015748,0.014458,0.026188,0.019122,0.026607,-0.032700,...,-0.006334,0.029853,0.000000,0.004409,0.000000,0.107414,-0.001409,0.023350,0.026857,0.026756
2000-01-07,0.003824,0.058848,-0.018065,0.024293,0.000000,-0.007203,0.009646,0.043918,0.006729,0.024625,...,-0.002058,-0.029853,-0.061748,-0.002937,-0.095310,0.049522,0.022316,-0.023350,-0.015508,0.015359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,-0.006431,0.000470,0.001360,-0.001997,0.001313,0.001462,0.026536,-0.003053,-0.006605,0.005277,...,0.005374,0.001432,-0.004289,-0.004009,-0.025318,-0.004172,0.000525,0.000539,-0.003762,0.000085
2019-12-24,0.000000,0.001761,0.002715,-0.004360,0.002621,-0.001462,-0.000491,0.008627,0.006365,0.002103,...,-0.012207,0.005706,-0.002447,0.008003,-0.025975,0.001936,-0.010208,0.002959,0.002836,0.004546
2019-12-26,-0.006473,-0.000352,-0.009192,-0.001655,0.002179,0.008741,-0.000164,0.002523,-0.006004,-0.005266,...,-0.006716,0.000000,-0.001860,-0.008003,0.051293,-0.002650,0.005469,-0.004577,0.001196,-0.000527
2019-12-27,0.000000,-0.003997,0.009869,-0.015136,-0.003926,0.003422,0.018980,0.003020,-0.002895,0.003914,...,0.018422,-0.007138,-0.001864,-0.011500,-0.025318,-0.007478,-0.007771,-0.004327,0.009151,-0.005592
